# Pydeck + Earth Engine: Points FeatureCollection

This is an example of using Google Earth Engine to visualize a `FeatureCollection` of points in [Pydeck](https://pydeck.gl). To install and run this notebook locally, refer to the [Pydeck Earth Engine documentation](https://earthengine-layers.com/docs/developer-guide/pydeck-integration).

To see this example online, view the [JavaScript version][js-example].

[js-example]: https://earthengine-layers.com/examples/power-plants

Import required packages:

In [ ]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import ee

### Authenticate with Earth Engine

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

### Power plants dataset

This example uses the [Global Power Plant Database][dataset], which contains location of power plants around the world, as well as metadata of which fuel source is used, and how much energy is created.

[dataset]: https://developers.google.com/earth-engine/datasets/catalog/WRI_GPPD_power_plants

First we'll render using EarthEngine's native capabilities, then we'll render using pydeck's rich data-driven styling capabilities.

In [ ]:
# Load the FeatureCollection
table = ee.FeatureCollection("WRI/GPPD/power_plants")

In [ ]:
# Create color palette
fuel_color = ee.Dictionary({
  'Coal': '000000',
  'Oil': '593704',
  'Gas': 'BC80BD',
  'Hydro': '0565A6',
  'Nuclear': 'E31A1C',
  'Solar': 'FF7F00',
  'Waste': '6A3D9A',
  'Wind': '5CA2D1',
  'Geothermal': 'FDBF6F',
  'Biomass': '229A00'
})

In [ ]:
# List of fuels to add to the map
fuels = ['Coal', 'Oil', 'Gas', 'Hydro', 'Nuclear', 'Solar', 'Waste',
    'Wind', 'Geothermal', 'Biomass']

In [ ]:
def add_style(point):
    """Computes size from capacity and color from fuel type.
    
    Args:
        - point: (ee.Geometry.Point) A Point
        
    Returns:
        (ee.Geometry.Point): Input point with added style dictionary
    """
    size = ee.Number(point.get('capacitymw')).sqrt().divide(10).add(2)
    color = fuel_color.get(point.get('fuel1'))
    return point.set('styleProperty', ee.Dictionary({'pointSize': size, 'color': color}))

In [ ]:
# Make a FeatureCollection out of the power plant data table
pp = ee.FeatureCollection(table).map(add_style)

In [ ]:
# Create a layer for each fuel type
layers = []
for fuel in fuels:
    layer = EarthEngineLayer(
        pp.filter(ee.Filter.eq('fuel1', fuel)).style(styleProperty='styleProperty', neighborhood=50),
        id=fuel,
        opacity=0.65,
    )
    layers.append(layer)

Then just pass this layer to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [ ]:
view_state = pdk.ViewState(latitude=36, longitude=-53, zoom=3)
r = pdk.Deck(
    layers=layers, 
    initial_view_state=view_state
)
r.show()

### Vector rendering

To experience the full potential of pydeck, we need to explore rendering the data with vector output. Rendering the data as a vector allows for client-side data driven styling and picking objects to get information on each.

In [ ]:
fuel_color = {
  'Coal': [0, 0, 0],
  'Oil': [89, 55, 4],
  'Gas': [188, 128, 189],
  'Hydro': [5, 101, 166],
  'Nuclear': [227, 26, 28],
  'Solar': [255, 127, 0],
  'Waste': [106, 61, 154],
  'Wind': [92, 162, 209],
  'Geothermal': [253, 191, 111],
  'Biomass': [34, 154, 0],
}

In [ ]:
layers = []
for fuel in fuels:
    layer = EarthEngineLayer(
        # Create each layer as consisting of a single fuel type
        table.filter(ee.Filter.eq('fuel1', fuel)),
        vis_params={},
        # Define a custom radius
        getRadius='.properties.capacitymw ** 1.35',
        lineWidthMinPixels=0.5,
        pointRadiusMinPixels=2,
        # Use dictionary for color mapping
        get_fill_color=fuel_color[fuel],
        # Provide a list of extra dataset properties that should be downloaded for rendering
        # Without this, only the geometry will be downloaded
        selectors=['capacitymw'],
        # Render as a vector, instead of as a raster generated on EE's servers
        as_vector=True,
        id=fuel,
        opacity=0.65,
    )
    layers.append(layer)

Then just as before, we define a `ViewState` and pass these layers to the `Deck` object.

In [ ]:
view_state = pdk.ViewState(latitude=36, longitude=-53, zoom=3)
r = pdk.Deck(
    layers=layers, 
    initial_view_state=view_state
)
r.show()